In [1]:
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from konlpy.tag import Kkma
from konlpy.tag import Hannanum
from konlpy.tag import Komoran
import xlrd
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from matplotlib import font_manager,rc
font_name = font_manager.FontProperties(fname='C:\Windows\Fonts\malgun.ttf').get_name()
rc('font',family=font_name)
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

# 데이터전처리

In [180]:
df=pd.concat([data_happy,data_anger,data_flutter,data_pleasure,data_regret,data_stable,data_sad,data_depress],ignore_index=True)
df.drop(['Unnamed: 0'],axis=1,inplace=True)
df

,content,label
0,★ 사랑합니다.⠀⠀Have a GODday🙏⠀핸드폰 배경화면 ...,1
1,🤬삐삐-삐-삐삐--프로필 사진이나 블로그 등 상업...,1
2,김호중 공식 팬카페 가입을 환영합니다 오시는 길은 프로필에#김호중 #kimhojoo...,1
3,김호중 공식 팬카페 가입을 환영합니다 오시는 길은 프로필에#김호중 #kimhojoo...,1
4,1 2 3 🎬📸📸 !!!😻🌲😽STRAY & STRONG 🇰🇷🤟🥵#안녕 #머...,1
...,...,...
15995,벼랑 끝에 서있는 것 같은 오늘.,6
15996,괜찮아질 때까지 마음껏 우울해하기...뒤에 타임랩스도 있음#artwork #draw...,6
15997,#우울#심란요즘 심란하고 힘들어서 읽기시작한 책.관계정리가 필요한 시기인듯합니다.,6
15998,나 힘든거 제일 잘 알았으면서. 제일 가까이서 지켜봤으면서. 내 마음 잘 이해한다고...,6


In [183]:
def prepocessing(data):
    for i in data.index:
        data['content']=data['content'].str.replace(pat=r'[^\w]',repl=r" ",regex=True)
        
    data.dropna(inplace=True)
    data.drop_duplicates(['content'],inplace=True)
    data.reset_index(inplace=True)
    data.drop(['index'],axis=1, inplace=True)


In [182]:
prepocessing(df)

In [184]:
df.drop(['index'],axis=1,inplace=True)

In [213]:
df['label'].value_counts()
df.to_excel("data/pre_data_set.xlsx")

In [419]:
df3=pd.read_excel("data/pre3_data_set.xlsx")

In [836]:
df5=pd.read_excel('data/pre_data_set.xlsx')


In [840]:
#중복된 키워드 있을경우 삭제 
def duplicated_keyword_del(df):
    text=["행복","신남","분노","우울","설렘","안정","슬픔","후회"]
    t_cnt=0
    index_cnt=0
    for data in df['content']:
        cnt=0
    
        for j in text:
        
            if j in data:
                cnt+=1
           
            
        if cnt>=2:

            df.drop(df['Unnamed: 0'][index_cnt],inplace=True)
            t_cnt+=1
    
        index_cnt+=1


In [842]:
df5['label'].value_counts()

1    1810
3    1774
2    1689
5    1610
6    1545
8    1534
7    1493
4    1343
Name: label, dtype: int64

In [844]:
df5.reset_index(inplace=True,drop=True)

In [845]:
df5

,index,Unnamed: 0,content,label
0,0,0,사랑합니다 Have a GODday 핸드폰 배경화면 ...,1
1,1,1,삐삐 삐 삐삐 프로필 사진이나 블로그 등 상업...,1
2,2,2,김호중 공식 팬카페 가입을 환영합니다 오시는 길은 프로필에 김호중 kimhojoo...,1
3,3,3,1 2 3 STRAY STRONG 안녕 머...,1
4,4,4,엄마한테 용돈 100만원 주고 욕먹는 아들 ㅋㅋㅋㅋ 큰 웃음 만들기 프로젝트 ...,1
...,...,...,...,...
12793,14713,14713,벼랑 끝에 서있는 것 같은 오늘,6
12794,14714,14714,괜찮아질 때까지 마음껏 우울해하기 뒤에 타임랩스도 있음 artwork draw...,6
12795,14715,14715,우울 심란요즘 심란하고 힘들어서 읽기시작한 책 관계정리가 필요한 시기인듯합니다,6
12796,14716,14716,나 힘든거 제일 잘 알았으면서 제일 가까이서 지켜봤으면서 내 마음 잘 이해한다고...,6


In [850]:
df5.drop(['index'],axis=1,inplace=True)

In [854]:
df5.drop(['Unnamed: 0'],axis=1, inplace=True)

In [856]:
df5['content'].str.contains(" ").replace("")

0        True
1        True
2        True
3        True
4        True
         ... 
12793    True
12794    True
12795    True
12796    True
12797    True
Name: content, Length: 12798, dtype: bool

In [857]:
okt=Okt()
cnt=0
for data in df5['content']:
    if(len(okt.morphs(data))<1):
        print(data)
        print(cnt)
        df.drop(df.index[cnt],inplace=True    
    cnt+=1

 
162
  
204
    
403
                              
443
   
1152
       
1423
      
1591
                
5694
          
7245
     
8156
           
9819
               
11502


In [859]:
df5.sort_values(['label'],inplace=True)
df5.reset_index(drop=True, inplace=True)

In [860]:
df5

,content,label
0,사랑합니다 Have a GODday 핸드폰 배경화면 ...,1
1,소고기 애호박 미음 1일차시작은 냠냠 잘 먹다가 더달라는 짜증인줄 알았는데반전 먹기...,1
2,까꿍,1
3,춘천 첫 러닝내 사랑과 함께 야간 러닝데이트 기록,1
4,_행뽁 아직도 너무 귀여워엄마빠 둘만 있는걸 못보는 내사랑,1
...,...,...
12793,분노 방송을안한다고,8
12794,감성 감정 글귀스타그램 글귀 진실 가사 노래 이성 인성 독서 또라이 용서 분노 절...,8
12795,아 ㅋㄱㅋㄱㅋㄱㅋㄲㅋㄱㅋㄱㅋㄱㅋㅋㅋㄱㅋㄱㅋㄱㅋㄱㅋㄱㅋㄱㅋㄱㅋㄱㅋㄱㅋㄱㅋㄱㅋㄱㅋㄱㅋㄱ...,8
12796,오늘의 꽃 2 11 멜리사꽃말 동정개화시기 늦여름 늦여름에 꽃을 피워낸다고 알...,8


In [861]:
df5.to_excel("data/pre4_data_set.xlsx")

### 데이터셋 불러오기

In [2]:
df5=pd.read_excel("data/pre4_data_set.xlsx")

In [3]:
df5.drop(["Unnamed: 0"],axis=1, inplace=True)

In [4]:
komo=Komoran()
kkma=Kkma()
okt=Okt()

### 토크나이저
- konlpy의 okt이용
- 불용어도 넣을수있음

In [6]:
#토크나이저 okt 분석기 사용 명사/영어/동사/숫자/형용사/감탄사//파편화되있는단어가있어 foreign도 추가해야됨 추출// 
#불용어는 우리가 검색한 단어만 추가
#토크나이저를 조정하면서 실험해봐도 될듯
def tokenizer(raw, pos=["Noun","Alpha","Verb","Adjective"], stopword=[]):
    return [
        word for word, tag in okt.pos(
            raw, 
            norm=True,   #
            stem=True    #어간추출
            )
            if len(word) > 1 and tag in pos and word not in stopword
        ]

In [100]:
pipe_model=make_pipeline(TfidfVectorizer(tokenizer=tokenizer), LogisticRegression())
##기본 TfidfVectorizer, 로지스틱레그레션                                                                                                                               

In [7]:
X_train,X_test,y_train,y_test=train_test_split(df5.content,df5.label,
                                                 test_size=0.3, random_state=23)

In [880]:
pipe_model.fit(X_train, y_train)

C:\Users\smhrd\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(tokenizer=<function tokenizer at 0x000001D314AD4DC0>)),
                ('logisticregression', LogisticRegression())])

In [881]:
cross_val_score(pipe_model,X_train, y_train, cv=5).mean()

C:\Users\smhrd\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\smhrd\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

0.8239457787071908

In [883]:
#불용어로 키워드를 제외했을때 약 0.6의 정확도를 보임
#불용어를 제외 했을때는
pipe_model.score(X_test,y_test)

0.8195197521301317

In [882]:
pipe_model.predict(text)

array([1, 3, 8, 6, 5, 7, 4, 2], dtype=int64)

In [884]:
pipe_model.predict_proba(text)

array([[9.99997397e-01, 4.37813412e-07, 2.38099144e-07, 6.09783867e-07,
        2.91842812e-07, 1.58344607e-07, 6.76941481e-07, 1.90030033e-07],
       [1.35656452e-10, 1.16406971e-09, 9.99999994e-01, 9.06800157e-10,
        1.19865643e-09, 9.15404035e-10, 9.66187048e-10, 9.11619544e-10],
       [3.64758786e-10, 4.80508838e-10, 8.47303046e-10, 2.91675053e-10,
        8.98208064e-10, 3.04066534e-10, 6.99376729e-10, 9.99999996e-01],
       [3.58538804e-07, 3.33524812e-07, 8.70958195e-07, 4.24155833e-06,
        6.11001777e-07, 9.99990679e-01, 6.38754716e-07, 2.26675845e-06],
       [3.93119671e-10, 1.92535001e-09, 1.09280250e-09, 1.19230666e-09,
        9.99999991e-01, 1.31172177e-09, 1.40785867e-09, 1.57637722e-09],
       [6.05342880e-10, 1.74648178e-09, 2.38848083e-09, 1.69473338e-09,
        1.70799658e-09, 1.50722982e-09, 9.99999988e-01, 2.47493235e-09],
       [3.08110926e-09, 1.38824001e-08, 9.73630538e-09, 9.99999929e-01,
        7.23115330e-09, 1.00152332e-09, 8.47384440e-09, 2.

In [753]:
pipe_model.predict(text)

array([4], dtype=int64)

### 단어별 가중치 맵핑

In [870]:
len(pipe_model.steps[0][1].vocabulary_)

27952

In [45]:
voca=final_model.steps[0][1]

In [47]:
len(voca.vocabulary_)

23704

In [48]:
logi_W=final_model.steps[1][1]

In [49]:
logi_W=logi_W.coef_


In [50]:
logi_W.shape

(8, 23704)

In [51]:
logi_W=logi_W.T

In [52]:
logi_W.shape

(23704, 8)

In [471]:
logi_W[0]

array([ 0.13696077, -0.66731934, -0.15693877, -0.35083457,  1.30026094,
        0.18976786, -0.23817623, -0.21372066])

In [53]:
df_model=pd.DataFrame([voca.vocabulary_.values(),voca.vocabulary_.keys()],index=['Index','Voca'])

In [54]:
df_model

,0,1,2,3,4,5,6,7,8,9,...,23694,23695,23696,23697,23698,23699,23700,23701,23702,23703
Index,4155,4328,4047,4159,4938,3098,3528,14310,7353,11402,...,6518,19450,3389,4993,12396,11210,13963,17503,13646,15281
Voca,아무도,없어서,신남,아빠,자유,를,봐,스타,그램,믹스견,...,건강 다이어트,졸업 하다,바람,저와,분노 에너지,문의 환영,소중하다 친구,의전,설레다 날씨,아침 7시


In [55]:
df_model=df_model.T

In [56]:
df_model

,Index,Voca
0,4155,아무도
1,4328,없어서
2,4047,신남
3,4159,아빠
4,4938,자유
...,...,...
23699,11210,문의 환영
23700,13963,소중하다 친구
23701,17503,의전
23702,13646,설레다 날씨


In [57]:
col=['coef:1','coef:2','coef:3','coef:4','coef:5','coef:6','coef:7','coef:8']

In [58]:
df_sorted=df_model.sort_values(by="Index")

In [59]:
df_sorted=df_sorted.reset_index(drop=True)

In [60]:
df_sorted

,Index,Voca
0,0,00
1,1,00 선팔
2,2,000
3,3,000 원
4,4,000원
...,...,...
23699,23699,𝑫𝑨𝒀
23700,23700,𝑫𝑨𝒀 5일
23701,23701,𝒅𝒐𝑻𝑬𝑹𝑹𝑨
23702,23702,𝒅𝒐𝑻𝑬𝑹𝑹𝑨 𝑩𝑶𝑮𝑶


In [61]:
final_df=pd.concat([df_sorted,pd.DataFrame(logi_W,columns=col)],axis=1)

In [65]:
final_df.sort_values(by="coef:7")

,Index,Voca,coef:1,coef:2,coef:3,coef:4,coef:5,coef:6,coef:7,coef:8
13627,13627,설레다,-3.513575,-2.226602,-2.844116,-2.341409,17.604074,-2.412523,-2.229935,-2.035914
14656,14656,신나다,-3.627250,-2.288174,17.980344,-2.378820,-2.564111,-2.599881,-2.193396,-2.328712
12371,12371,분노,-2.942732,-2.660987,-2.257895,-2.441150,-2.343118,-2.874167,-2.188468,17.708518
22950,22950,행복,14.175190,-1.946615,-2.559628,-2.217979,-2.857324,-0.416373,-2.055046,-2.122225
23443,23443,후회,-2.790220,16.258265,-2.088868,-2.585693,-2.266081,-2.190549,-2.038689,-2.298165
...,...,...,...,...,...,...,...,...,...,...
10285,10285,마음,-0.164406,-0.198368,-1.427956,-0.210243,0.150790,0.032522,2.085391,-0.267731
23532,23532,휴식,-0.617806,-0.416761,-0.226768,-0.387757,-0.397855,0.398245,2.146293,-0.497591
16674,16674,오일,-0.374402,-0.389840,-0.355937,-0.206980,-0.312867,-0.428634,2.467273,-0.398612
4213,4213,안정,-1.192822,-1.070505,-0.989393,-1.021416,-1.060707,-1.215154,7.526055,-0.976058


### 텍스트의 가중치 찾기

In [97]:
def findcoef(text):
    emot=final_model.predict(text)
    emot_coef=[]
    emot_co="coef:"+str(emot[0])
    for token in tokenizer(text[0]):
        df_mask=final_df['Voca']==token
        sel_df=final_df[df_mask]
        emot_po=sel_df[emot_co].values
        emot_coef.append([token,emot_po])
        
        emot_df=pd.DataFrame(emot_coef,columns=["단어","가중치"+emot_co])
        emot_df.drop_duplicates(['단어'],keep='last',inplace=True)
        emot_df.sort_values(by="가중치",ascending=False,inplace=True)
    return emot_df

In [737]:
pipe_model.predict_log_proba(text)

array([[-2.84087015, -2.40093182, -2.71104158, -0.67082082, -3.35954453,
        -2.3044361 , -3.12068807, -2.35870612]])

In [98]:
emot=pipe_model.predict(text)
type(emot)

NotFittedError: The TF-IDF vectorizer is not fitted

In [ ]:
df_emot=findcoef(text)

In [ ]:
pipe_model.predict_log_proba()

### 그리드서치

from sklearn.model_selection import GridSearchCV
param_grid={
    'tfidfvectorizer__max_df':[7000,9000,11000],
    'tfidfvectorizer__min_df':[7,9,11],
    'tfidfvectorizer__ngram_range':[(1,1),(1,2),(1,3)],
    'logisticregression__C':[0.01,0.1,1,10,100]
}
grid=GridSearchCV(pipe_model,param_grid,cv=5)

grid.fit(X_train,y_train)

grid.best_params_

grid.best_score_

In [8]:
########그리드 서치 바탕으로 만들 모델
final_model=make_pipeline(TfidfVectorizer(tokenizer=tokenizer,max_df=5000,min_df=3,ngram_range=(1,2)),
                         LogisticRegression(C=1))

In [9]:
final_model.fit(X_train,y_train)

C:\Users\smhrd\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(max_df=5000, min_df=3, ngram_range=(1, 2),
                                 tokenizer=<function tokenizer at 0x000001E9BD075A60>)),
                ('logisticregression', LogisticRegression(C=1))])

In [ ]:
final_model.score(X_test,y_test)

In [10]:
def makelist(text):
    textlist=[text]
    return textlist

## label
- 행복1, 신남3, 설렘5, 안정7 
- 후회2, 슬픔4, 우울6, 분노8

In [119]:
final_model.predict(makelist(text))

array([1], dtype=int64)

# flask이용 통신

In [11]:
import pickle # 데이터 저장/로딩을 위한 모듈
import pandas as pd
import numpy as np

In [12]:
from flask import Flask, redirect, request, Response, session

In [ ]:
app = Flask(__name__) # 내장변수 name을 이용해 서버를 구동시키는 객체 생성

@app.route("/predict", methods=["GET",'POST'])
def predict() :
    
    if request.method == 'POST':
        
        req=request.form
        content=req.get("content")
        #userid=req.get("id")
        emote=req.get("emote")
        date=req.get("date")
        print(date)
        print(emote)
        
        title=req.get("title")
        
        
        #데이터를 리스트화 시켜서 
        #학습된 모델에 예측
        predata=makelist(content)
        result=final_model.predict(predata)
        
        if result==1:
            analysis_e="행복"
        elif result ==2:
            analysis_e="후회"
        elif result ==3:
            analysis_e="신남"
        elif result ==4:
            analysis_e="슬픔"
        elif result ==5:
            analysis_e="설렘"
        elif result ==6:
            analysis_e="우울"
        elif result ==7:
            analysis_e="평온"
        elif result ==8:
            analysis_e="분노"
  
    return redirect("http://localhost:8086/PROJECT_2ND/WriteCon?title="+title+"&content="+content+"&emote="+emote+"&date="+date+"&analysis_e="+analysis_e)
    
    

if __name__ == "__main__" : # .py 파일에서 main함수 역할
    app.run(host="localhost", port="9000")

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:9000/ (Press CTRL+C to quit)


### 다중분류 모델

In [67]:
other_model=make_pipeline(TfidfVectorizer(tokenizer=tokenizer,max_df=5000,min_df=5,ngram_range=(1,2)), LogisticRegression(multi_class='multinomial',solver='lbfgs',C=1))

In [68]:
other_model.fit(X_train, y_train)

C:\Users\smhrd\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(max_df=5000, min_df=5, ngram_range=(1, 2),
                                 tokenizer=<function tokenizer at 0x0000029C527830D0>)),
                ('logisticregression',
                 LogisticRegression(C=1, multi_class='multinomial'))])

In [69]:
other_model.score(X_test,y_test)

0.8270833333333333

In [82]:
other_model.predict_proba(text)

array([[0.0157049 , 0.00632434, 0.00491311, 0.00821446, 0.00635089,
        0.64092018, 0.31052359, 0.00704853]])